# Plot misincorporation heatmaps from BAM files

This short pipeline makes pileups from BAM files, extract the pileups in sequences of interest, and build misincorporation heatmaps as output.

In [ ]:
# Following script saved as parse_pileups.py
# Dependency of pileup_analysis.sh

import sys
import re

infile = open(sys.argv[1], 'rt')
infile.readline()
outfile = open(sys.argv[2], 'w+')

# List sequences of interests ID from custom reference
seq_of_interest = ['ID1','ID2','ID3']

# Extract relevant information (Depth, Allele Freqs) from pileup files for sequences of interest only
for line in infile:
    if line[0] != '#':
        array = line.strip().split('\t')
        ID, position, ref, alt, INFO = array[0], array[1], array[3], array[4], array[7]

        if ID in seq_of_interest:
            alt = alt.split(',')
            x = re.search("DP=([0-9]+);AD=([0-9,\,]+)", INFO)
            DP, AD = x.group(1), x.group(2)
            AD = AD.split(',')

            outfile.write(ID+'\t'+position+'\t'+DP+'\t'+ref+'\t'+str(float(AD[0])/float(DP))+'\t')
            
            for i in range(0, len(alt)):
                outfile.write(alt[i]+'\t'+str(float(AD[i+1])/float(DP))+'\t')

            outfile.write('\n')
            
            
            

In [ ]:
%%bash

# Following script saved as pileup_analysis.sh
# Command: sbatch pileup_analysis.sh experiment
# Create the pileups and filter output for sequences of interest

###################################################################################################################
#! /bin/bash

#SBATCH --job-name wil_works
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=4
#SBATCH --partition=ccr
#SBATCH --time=24:00:00
#SBATCH --gres=lscratch:500 
#SBATCH --mem=50g

experiment=$1

# Make pileups
module load samtools
for file in ${experiment}.splitFastq/*.sorted.bam ; do bcftools mpileup -d 1000000000  ${file} -f custom_reference.fa -a INFO/AD > ${file}.pileup; done

# Keep only sequences of interest
for file in ${experiment}.splitFastq/*.pileup ; do python3 parse_pileups.py ${file} ${file}.out ; done

# Rename files
for file in ${experiment}.splitFastq/*.pileup.out ; do mv "${file}" "`echo ${file} | sed 's/.sorted.bam//'`"; done

###################################################################################################################





In [ ]:
%%bash

# Run previous pipeline
sbatch pileup_analysis.sh experiment_dir


In [ ]:
# summarize_pileups.py
# gather all pileups in one table - useful for publication
# https://www.geeksforgeeks.org/how-to-read-multiple-text-files-from-folder-in-python/

import os
import sys
import pandas as pd

# Folder Path 
path = sys.argv[1]

# Change the directory 
os.chdir(path) 

all_rows = {}

# Read text files
def read_text_file(file_path): 
    with open(file_path, 'r') as f: 

        rows = {}
        
        for line in f:
            
            array = line.strip().split('\t')
            ID = array[0]+'|'+array[1]
            DP = array[2]

            variants = ''
            
            for field in array[3:]:
                if field != '<*>':
                    variants = variants+field+','
                else:
                    break

            if ID not in rows:
                rows[ID] = ''

            rows[ID] = rows[ID] + 'DP='+DP+','+ variants
            
    return(rows)

                
# iterate through all files
for file in os.listdir(): 
    # Check whether file is in text format or not 
    if file.endswith("pileup.out"):
    
        column = file.replace(".pileup.out", "")

        file_path = f"{path}{file}"
  
        # call read text file function 
        all_rows[column] =  read_text_file(file_path)

        
final_rows = {}

columns = []
for column in all_rows:
    columns.append(column)
    
    for row in all_rows[column]:
        if row not in final_rows:
            final_rows[row] = []
            
        final_rows[row].append(all_rows[column][row])
        
df = pd.DataFrame.from_dict(final_rows, orient='index', columns=columns)
df.to_csv('pileup_summary.csv')




In [ ]:
%%bash

# Run previous script
cd experiment_dir
python3 summarize_pileups.py ./
cd ..

In [ ]:
# misincorporation_heatmap.py
# prepare input for heatmap

import pandas as pd
import numpy as np
import sys

translate = {'ID1' : 'SEQUENCE_1', 'ID2' : 'SEQUENCE_2', 'ID3' : 'SEQUENCE_3'}

infile = pd.read_csv('pileup_summary.csv')

ID_POS = infile.iloc[:, 0]

experiment = sys.argv[1]

# extract 3 replicates of the same experiment 
Rep1 = infile[experiment+'_rep1']
Rep2 = infile[experiment+'_rep2']
Rep3 = infile[experiment+'_rep3']

dic_depths = {}
dic = {}

# Compute freq of incorporations matching the reference
# Also returns read depth
for i in range(0, len(ID_POS)):
    
    ID, POS = ID_POS[i].split('|')
    
    DP_depths = []
    DP_refs = []
    
    for Rep in [Rep1, Rep2, Rep3]:
        
        if 'nan' in str(Rep[i]):
            
            DP_depths.append(np.nan)
            DP_refs.append(np.nan)
                
        else:
            DP = Rep[i].split(',')[0].split('=')[1]
            
            DP_ref = Rep[i].split(',')[2]

            if int(DP) > 1:
                DP_refs.append(float(DP_ref))
                DP_depths.append(int(DP))
            else:
                DP_refs.append(np.nan)  
                DP_depths.append(np.nan)
    
    DP_ref_mean = np.nanmean(DP_refs)
    
    ID = translate[ID]
    
    if ID not in dic:
        dic_depths[ID] = []
        dic[ID] = []
        
    dic_depths[ID].append(DP_depths)
    dic[ID].append(DP_ref_mean)
    

df = pd.DataFrame.from_dict(dic, orient='index')
df.to_csv(experiment+'_HeatMap.csv')

df_depths = pd.DataFrame.from_dict(dic_depths, orient='index')
df_depths.to_csv(experiment+'_HeatMap_depths.csv')






In [ ]:
%%bash

# Run previous scripts for 3 different conditions
for experiment in Condition_1 Condition_2 Condition_3 ; do python3 misincorporation_heatmap.py ${experiment} ; done



In [ ]:
%%R


##########################################################################################
# https://www.r-bloggers.com/2015/09/passing-arguments-to-an-r-script-from-command-lines/
# heatmap.R
# Plot heatmap

library(ggplot2)
library(heatmaply)
library(plotly)
library(orca)


args = commandArgs(trailingOnly=TRUE)


# test if there is at least one argument: if not, return an error
if (length(args)==0) {
  stop("At least one argument must be supplied (input file).n", call.=FALSE)
}



data <- read.csv(args[1], header=TRUE)
data <- cbind(data[,1], data[,2:76])
colnames(data) <- c('ID', seq(1,length(data[1,])-1))

toplot <- data[-1]
rownames(toplot) <- data$ID
toplot <- toplot[order(rownames(toplot)), ]


p <- heatmaply(toplot, 
        dendrogram = "none",
        file = paste0(args[1],'.pdf'),
        #plot_method = "plotly",
        xlab = "", ylab = "", 
        main = args[1],
        #scale = "row",
        margins = c(60,100,40,20),
        grid_color = "white",
        grid_width = 0.00001,
        titleX = FALSE,
        hide_colorbar = TRUE,
        branches_lwd = 0.1,
        fontsize_row = 5, fontsize_col = 5,
        labRow = rownames(toplot),
        column_text_angle = 90,
        )
#############################################################################################



In [ ]:
%%bash

for experiment in Condition_1 Condition_2 Condition_3 ; do Rscript heatmap.R ${experiment}_HeatMap.csv ; done
